In [1]:
import os
import json
from dotenv import load_dotenv
from ibm_watson_machine_learning.foundation_models import Model
from ibm_watson_machine_learning.metanames import GenTextParamsMetaNames as GenParams


In [2]:
API_KEY=''
IBM_CLOUD_URL=''
PROJECT_ID=''

In [3]:
#config Watsonx.ai environment
load_dotenv()
api_key = API_KEY
ibm_cloud_url = IBM_CLOUD_URL
project_id = PROJECT_ID
if api_key is None or ibm_cloud_url is None or project_id is None:
    raise Exception("Ensure you copied the .env file that you created earlier into the same directory as this notebook")
else:
    creds = {
        "url": ibm_cloud_url,
        "apikey": api_key 
    }

In [9]:
def send_to_watsonxai(prompts,
                    model_name="google/flan-ul2",
                    decoding_method="greedy",
                    max_new_tokens=200,
                    min_new_tokens=30,
                    temperature=1.0,
                    repetition_penalty=1.0
                    ):
    '''
   helper function for sending prompts and params to Watsonx.ai
    
    Args:  
        prompts:list list of text prompts
        decoding:str Watsonx.ai parameter "sample" or "greedy"
        max_new_tok:int Watsonx.ai parameter for max new tokens/response returned
        temperature:float Watsonx.ai parameter for temperature (range 0>2)
        repetition_penalty:float Watsonx.ai parameter for repetition penalty (range 1.0 to 2.0)

    Returns: None
        prints response
    '''

    assert not any(map(lambda prompt: len(prompt) < 1, prompts)), "make sure none of the prompts in the inputs prompts are empty"

    # Instantiate parameters for text generation
    model_params = {
        GenParams.DECODING_METHOD: decoding_method,
        GenParams.MIN_NEW_TOKENS: min_new_tokens,
        GenParams.MAX_NEW_TOKENS: max_new_tokens,
        GenParams.RANDOM_SEED: 42,
        GenParams.TEMPERATURE: temperature,
        GenParams.REPETITION_PENALTY: repetition_penalty,
    }


    # Instantiate a model proxy object to send your requests
    model = Model(
        model_id=model_name,
        params=model_params,
        credentials=creds,
        project_id=project_id)

    output = []
    for prompt in prompts:
        o = model.generate_text(prompt)
        output.append(o)
        # print(o)
    return output

In [16]:
llama2c = "meta-llama/llama-2-70b-chat"
llama2 = "meta-llama/llama-2-70b"
granite="ibm/granite-13b-chat-v1"

# CLassification Zero shot learning

In [36]:
text = '''
ผมมีสัญญาเงินกู้บ้านกับ Wells Fargo มาหลายปีแล้ว จ่ายทุกเดือนเร็วอย่างน้อย 7-10 วัน ระหว่าง XX/XX/XXXX และ XX/XX/XXXX ผมจ่าย {$3000.00} ทุกเดือน 
ในปี XXXX ผมได้รับโทรศัพท์จาก Wells Fargo ว่าการชำระเงินรายเดือนของฉันไม่ถูกต้อง หลังจากการสนทนายาวนาน ฉันยอมจ่ายเพิ่มเติม {$750.00} 
เพื่อทำให้บัญชีปัจจุบันและชำระ $ XXXX ต่อไป ในวันที่ XX/XX/XXXX ฉันได้รับจดหมายจาก XXXX 
ว่าสัญญาเงินกู้ของฉันอยู่ในสถานะเริ่มต้นและแนะนำให้ฉันดำเนินการทันที หลังจากการสนทนายาวนาน 
ฉันคุ้นเคยว่าในวันที่ XX/XX/XXXX ธนาคารได้รับการชำระเงินของฉันเช่นเคย แต่เนื่องจากจำนวนเงินน้อยกว่าที่พวกเขาต้องการ 
พวกเขาไม่ได้จ่ายสัญญาเงินกู้ของฉัน แต่พวกเขาได้นำจำนวนเงินทั้งหมดไปชำระเงินต้น พวกเขาไม่เคยบอกฉัน 
พวกเขาได้รายงานข้อมูลของฉันให้กับหน่วยงานเครดิต 
'''

## Sentiment analysis

In [18]:
#Q1 Code - enter prompt and parameters in this cell

system_prompt = """
You are the sentiment analysis program which return json format {"result":"positive"} or {"result": "negative"}, 
You must answer only json format withou any explanation, 
Output Schema: schema = {
      "type": "object",
      "properties": {
        "result": "string"
            }
        }
        """
review = text
user_message = f"Please do sentiment analysis of this review for me ```{review}```"

llama2_prompt = f"""
    <s>[INST] <<SYS>>
        {{
        {system_prompt}
        }}
    <</SYS>>
        {{
        {text}
        }}
        [/INST]
"""
#Complete your prompt here 

response = send_to_watsonxai(prompts=[llama2_prompt],
                            model_name=granite
                            )

responseC = send_to_watsonxai(prompts=[llama2_prompt],
                            model_name=llama2c
                            )

# Remember to try changing the model and/or using non-default parameters to achieve your goal when prompting isn't enough
# response = send_to_watsonxai(prompts=[prompt], model_name="google/flan-ul2", decoding_method="greedy", max_new_tokens=100,
#                              min_new_tokens=30, temperature=1.0, repetition_penalty=2.0)


In [21]:
print(response)
print(responseC)

['        ]\n        <s>\n{"result": "negative"}</s>\n        ]\n        <s>\n{"result": "negative"}</s>\n        ]<|endoftext|>']
['        {\n  "result": "negative"\n}\n\nExplanation:\n\nThe text expresses frustration and disappointment with Wells Fargo\'s handling of the person\'s loan. The person mentions that they have been making payments on time for many years, but Wells Fargo has not been applying the payments correctly, causing the person\'s account to be in arrears. The person also mentions that they have been trying to resolve the issue']


## Classification

In [37]:
instruction= '''
บทความต่อไปนี้เป็นการร้องเรียนของผู้บริโภค 
การร้องเรียนเกี่ยวกับหนึ่งในตัวเลือกเหล่านี้: บัตรเครดิต, รายงานเครดิต, สินเชื่อที่จดทะเบียนและสินเชื่อ, ธนาคารขายปลีก, หรือการค้าสินทรัพย์ 
โปรดอ่านย่อด้านล่างและกำหนดว่าการร้องเรียนนี้เกี่ยวข้องกับตัวเลือกใด.
'''


system_prompt = """
You are the sentence topic classifier can understand Thai.
Given only 5 classes that you have to classify the input sentence: [credit cards, credit reporting, mortgages and loans, retail banking, debt collection].
Example output: ['mortgages and loans']
Example output: ['credit cards']
Example output: ['credit reporting']
Example output: ['retail banking']
Example output: ['debt collection']
Classify the input sentence into only 1 class less than 10 words


        """
user_message = f"Please do classify review for me ```{text}```"

llama2_prompt = f"""
    <s>[INST] <<SYS>>
        {{
        {system_prompt}
        }}
    <</SYS>>
        {{
        {text}
        }}
        [/INST]
"""

response = send_to_watsonxai(prompts=[llama2_prompt],
                            model_name=granite
                            )

responseC = send_to_watsonxai(prompts=[llama2_prompt],
                            model_name=llama2c
                            )

In [39]:
# print(response)
print(responseC)

["        Based on the given input sentence, the topic can be classified into the following category:\n\n* Mortgages and loans\n\nThe sentence discusses the person's experience with a mortgage loan from Wells Fargo, including the loan amount, payment schedule, and issues with the loan servicer. Therefore, it falls under the category of mortgages and loans."]


# Oneshot learning

In [51]:
text = '''
วันนี้ฉันจ่ายเงิน 500 บาท ที่ธนาคารกรุงไทย โดยใช้หมายเลขโทรศัพท์ 099221xxyy ให้กับนายชนาธิ ณัฐวุฒิ
'''

In [52]:
system_prompt = """
You are the sentence entity extractor that can understand Thai.
You have to follow the example
        """

example_entity = """
sentence: ฉันถอนเงิน 100 บาท จากธนาคารไทยพาณิชย์ ด้วยเบอร์ 0649611111 สมชาย
entity: {
  "Amount": "100 บาท",
  "Bank Name": "ธนาคารไทยพาณิชย์",
  "Phone Number": "0649611111",
  "Name": "สมชาย"
}

"""




user_message = f"Please do extract entities for me ```{text}```"

llama2_prompt = f"""
    <s>[INST] <<SYS>>
        {{
        {system_prompt}
        EXAMPLE:
        {example_entity}
        }}
        
    <</SYS>>
        {{
        {text}
        }}
        [/INST]
"""

response = send_to_watsonxai(prompts=[llama2_prompt],
                            model_name=granite
                            )

responseC = send_to_watsonxai(prompts=[llama2_prompt],
                            model_name=llama2c
                            )

In [53]:
print(responseC)
print(" ")
print(response)

['        {\n  "Amount": "500 บาท",\n  "Bank Name": "ธนาคารกรุงไทย",\n  "Phone Number": "099221xxyy",\n  "Name": "นายชนาธิ ณัฐวุฒิ"\n}']
 
['        {\n        "Amount": "500 บาท",\n        "Bank Name": "กรุงไทย",\n        "Phone Number": "099221xxyy",\n        "Name": "นายชนาธิ"\n}<|endoftext|>']


## Two shot learning

In [72]:
text2 = """ สวัสดี ฉันชื่อ ราวี ดูเบ ฉันมาเขียนถึงคุณเพื่อรายงานรายการทำรายการที่ไม่ได้รับอนุญาตบนบัตรเครดิตของฉันครับ
เมื่อวันที่ 30 มีนาคม 2023 ฉันสังเกตรายการชาร์จ 1,000 ดอลลาร์ในรายการบัตรเครดิตของฉันที่ไม่ได้รับอนุญาตในวันนั้น ธุรกรรมนี้ทำการที่ร้านอาหารในนิวยอร์ก 
ในขณะที่ฉันอยู่ในแคลิฟอร์เนียในวันนั้นครับ ฉันกังวลเกี่ยวกับความปลอดภัยของบัญชีของฉันและฉันคิดว่านี่ควรจะได้รับการตรวจสอบโดยด่วนครับ
โปรดติดต่อฉันที่หมายเลขโทรศัพท์ 123-456-7890 หรืออีเมลที่ ravi.dube@email.com เพื่อให้ข้อมูลปรับปรุงเกี่ยวกับการสืบสวนครับ
ธนาคาร Tisco หมายเลขบัตรเครดิตของฉันคือ 3572267594198019 และหมายเลขประจำตัวประชาชนของฉันคือ 175-43-9027 ฉันยังคาดหวังที่จะได้รับข้อมูลจากคุณเร็ว ๆ นี้ครับ
"""


In [73]:
example_entity2 = """
sentence: ขอบคุณที่ติดต่อกับฉันครับ บัตรเครดิตของฉันถูกยกเลิกเมื่อฉันอยู่ในต่างประเทศเป็นวันหยุดครับ หมายเลขบัตรเครดิตของฉันคือ 39482374859 และชื่อบัญชีของฉันคือ ฟิล สมิธ และที่อยู่ของฉันคือ 34 ซิตีเวล, เมลเบิร์น, 3000 วิซซาดีครับ วิธีการติดต่อที่ฉันชอบคืออีเมล: phil.smith@email.com ครับ
entity: {
  "Credit Card Number": "39482374859",
  "Account Name": "ฟิล สมิธ",
  "Address": "34 ซิตีเวล, เมลเบิร์น, 3000 วิซซาดี",
  "Email": "phil.smith@email.com"
}
"""

In [74]:
user_message = f"Please do extract entities for me ```{text2}```"

system_prompt = """
You are the sentence entity extractor that can understand Thai.
You have to follow the output JSON format
        """

llama2_prompt = f"""
    <s>[INST] <<SYS>>
        {{
        {system_prompt}
        EXAMPLE1:
        {example_entity}
        EXAMPLE2:
        {example_entity2}

        }}
        
    <</SYS>>
        {{
        sentence:
        {text2}
        entity:
        }}
        [/INST]
"""

response = send_to_watsonxai(prompts=[llama2_prompt],
                            model_name=granite
                            )

responseC = send_to_watsonxai(prompts=[llama2_prompt],
                            model_name=llama2c,
                              max_new_tokens=250
                            )

In [75]:
print('Granite')
print(response)
print('llama')
print(responseC)

Granite
['        {\n        "Name": "สวัสดี",\n        "Amount": "1,000 ดอลลาร์",\n        "Bank Name": "Tisco",\n        "Phone Number": "3572267594198019",\n        "Email": "ravi.dube@email.com"\n}<|endoftext|>']
llama
['    {\n  "Amount": "1,000 ดอลลาร์",\n  "Bank Name": "ธนาคาร Tisco",\n  "Phone Number": "123-456-7890",\n  "Email": "ravi.dube@email.com",\n  "Credit Card Number": "3572267594198019",\n  "National ID Number": "175-43-9027"\n}\n\nIs there anything else I can help you with?']
